# T5. REFERENCE LEVELS

In this notebook, we extract the reference levels from the t1_semantic segmentation/ t2_instance segmentation

## LIBRARIES

In [20]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUTS

In [5]:
name='beton_labels'

path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}.laz')
class_file=path/'_classes.json'

name=name.split('_')[0]
json_output_path=os.path.join(path,f'{name}_levels.json') 
geometry_output_path= os.path.join(path,f'{name}_levels.obj') # these are the bounding surfaces of the reference levels (optional)


Import Classes

In [6]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import PCD

In [8]:
laz=laspy.read(pcd_input_path)

In [9]:
pcdNodes=[]
#split pcd per object
for i in np.unique(laz['classes']):
    idx=np.where(laz['classes']==i)
    points=laz.xyz[idx]
    # colors=np.array([laz.red[idx],laz.green[idx],laz.blue[idx]])
    object_labels=laz['objects'][idx]

    class_obj=next((class_obj for class_obj in json_data['classes'] if float(class_obj['id']) ==i), json_data['classes'][0])
    class_name=class_obj['name']

    # pcd.colors=o3d.utility.Vector3dVector(colors)
    for j in np.unique(object_labels):
        
        new_points=points[np.where(object_labels==j)]
        if new_points.shape[0]>100:
            pcd=o3d.geometry.PointCloud()
            pcd.points=o3d.utility.Vector3dVector(new_points)

            pcdNodes.append(PointCloudNode(resource=pcd,
                                        class_id=i,
                                        object_id=j,
                                        color=ut.random_color(),
                                            name=class_name+f'_{j}'))

print(f'{len(pcdNodes)} pcdNodes created!')

85 pcdNodes created!


In [10]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd])

## PROCESSING

Group floor point clouds

In [11]:
#retrieve floor points
floorNodes=[n for n in pcdNodes if n.class_id ==0]
print(f'{len(floorNodes)} floorNodes detected!')
for n in floorNodes:
    print(f' {n.name} with {n.orientedBoundingBox}')


9 floorNodes detected!
 Floors_1 with OrientedBoundingBox: center: (103783, 194671, 9.37336), extent: 3.51333, 1.64835, 0.095465)
 Floors_2 with OrientedBoundingBox: center: (103778, 194657, 9.15707), extent: 16.7819, 2.46913, 0.119764)
 Floors_3 with OrientedBoundingBox: center: (103777, 194666, 9.19541), extent: 7.47256, 1.46945, 0.0813937)
 Floors_4 with OrientedBoundingBox: center: (103788, 194655, 9.15462), extent: 9.95826, 1.36459, 0.236444)
 Floors_5 with OrientedBoundingBox: center: (103793, 194660, 9.2283), extent: 4.40512, 1.33574, 0.0819982)
 Floors_6 with OrientedBoundingBox: center: (103797, 194663, 9.2351), extent: 4.50277, 1.18646, 0.0739727)
 Floors_7 with OrientedBoundingBox: center: (103793, 194670, 9.2808), extent: 17.4479, 6.25487, 0.348126)
 Floors_8 with OrientedBoundingBox: center: (103787, 194663, 9.12218), extent: 20.6706, 15.8668, 0.262362)
 Floors_9 with OrientedBoundingBox: center: (103788, 194672, 9.10797), extent: 6.77982, 5.34224, 0.116553)


In [12]:
# for now, put all floorNodes in a single group (1 reference level)
groupedFloorNodes=[floorNodes]
print(groupedFloorNodes)


[[<geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005E710>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005E470>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005E950>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005FB50>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005D960>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005D810>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005EDA0>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005FF70>, <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000234C005EB90>]]


Compute reference Planes

In [13]:
#compute height underneight all floor points of a level (according to the lowest slab)
referenceNodes= []

for i,nodes in enumerate(groupedFloorNodes):
    #create sessionNode
    referenceNode=SessionNode(linkedNodes=nodes,
                                name=str(i)+'0')
    
    #determine height -> note that this can be negative
    weights=[float(len(np.asarray(n.resource.points))) for n in nodes]
    heights= [float(n.cartesianTransform[2,3]) for n in nodes]
    weighted_height= np.average(heights, weights=weights)

    #compute plane from cornerpoints orientedbounding box
    vertices=np.array([np.hstack((referenceNode.orientedBounds[0][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[1][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[2][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[4][0:2],weighted_height))])#,
    vertices=o3d.cpu.pybind.utility.Vector3dVector(vertices)
    triangles=o3d.cpu.pybind.utility.Vector3iVector(np.array([[0,1,2],[2,1,3]]))
    plane=o3d.geometry.TriangleMesh(vertices,triangles)

    #assign information to referenceNode
    referenceNode.plane=plane
    referenceNode.height=weighted_height
    
    referenceNodes.append(referenceNode)
    print(f' Level {referenceNode.name} created at height {referenceNode.height}')
# print(f'{len(referenceNodes)} referenceNodes created!')

 Level 00 created at height 9.149439581650363


In [14]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd,referenceNodes[0].plane])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 


In [15]:
{key:value for key, value in referenceNode.__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_linkedNodes': [<geomapi.nodes.pointcloudnode.PointCloudNode at 0x234c005e710>,
 '_linkedSubjects': [rdflib.term.URIRef('file:///Floors_1'),
  rdflib.term.URIRef('file:///Floors_2'),
  rdflib.term.URIRef('file:///Floors_3'),
  rdflib.term.URIRef('file:///Floors_4'),
  rdflib.term.URIRef('file:///Floors_5'),
  rdflib.term.URIRef('file:///Floors_6'),
  rdflib.term.URIRef('file:///Floors_7'),
  rdflib.term.URIRef('file:///Floors_8'),
  rdflib.term.URIRef('file:///Floors_9')],
 '_subject': rdflib.term.URIRef('file:///fc8e4412-eaaf-11ee-ace5-a86daaa4a627'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': '00',
 '_cartesianBounds': array([1.03771596e+05, 1.03801558e+05, 1.94649773e+05, 1.94678088e+05,
        8.96833629e+00, 9.57155701e+00]),
 '_orientedBounds': array([[1.03784319e+05, 1.94647839e+05, 8.84432081e+00],
        [1.03803344e+05, 1.94665206e+05, 8.93121616e+00],
        [1.03770909e+05, 1.94662529e+05, 8.90041993e+00],
        [1.03784318e+05, 1.94647836e+05, 9

Compute Json information

In [21]:
for n in referenceNodes:
    n.box=pcd.get_oriented_bounding_box()
    n.box.color=[1,0,0]
    
    rotation_matrix=copy.deepcopy(n.box.R) #! can these angles be negative?
    r =  Rotation.from_matrix(np.asarray(rotation_matrix))
    n.rotations = r.as_euler("zyx",degrees=True)

## EXPORT

geometry

In [22]:
joined_references=gmu.join_geometries([n.resource for n in referenceNodes])
success=o3d.io.write_triangle_mesh(filename=geometry_output_path, mesh=joined_references)
print(f' Saving joint references : {success}')
    

 Saving joint references : True


json with reference heights

In [26]:
#declare json
json_data = {
        "filename": ut.get_filename(geometry_output_path),
        "objects": []
    }

In [27]:
#fill json
for n in referenceNodes:
    obj = {
            "name": 'level_'+n.name,
            "centroid": {
                "x": n.box.center[0],
                "y": n.box.center[1],
                "z": n.height
            },
            "dimensions": {
                "length": n.box.extent[0],
                "width": n.box.extent[1],
                "height": n.height
            },
            "rotations": {
                "x": 0,
                "y": 0,
                "z": n.rotations[0]
            }
            }
    json_data["objects"].append(obj)

In [28]:
#write this information to the 3D detection json
with open(json_output_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)
print("JSON data written to file:", json_output_path)

JSON data written to file: c:\Users\Maarten\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\beton_levels.json
